In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reload modules to get latest changes
import importlib
import params
import agents
import model

params = importlib.reload(params)
agents = importlib.reload(agents)
model = importlib.reload(model)

from model import NewsVendorModel


Computing optimal benchmarks (this may take a moment)...


NameError: name 'C' is not defined

In [ ]:
# Configure partner prediction mode
params.USE_PARTNER_PREDICTION = False  # False = basic, True = sophisticated

# Run simulation with greedy agents
agent_types = ['greedy', 'greedy', 'greedy']
model = NewsVendorModel(agent_type=agent_types)

for i in range(params.ROUNDS):
    model.step()

# Pull data 
model_df = model.datacollector.get_model_vars_dataframe()
agent_df = model.datacollector.get_agent_vars_dataframe()

In [ ]:
# View agent data
agent_df

In [ ]:
# View model data
model_df

In [ ]:
# Separate price and quantity data
df = agent_df.reset_index()

price_data = df[df["AgentID"] == 1][["Step", "Action", "Reward"]].rename(columns={"Action": "Price"})
qty_data = df[df["AgentID"] == 2][["Step", "Action"]].rename(columns={"Action": "Quantity"})

combined_df = pd.merge(price_data, qty_data, on="Step")
combined_df = pd.merge(combined_df, model_df, left_on="Step", right_index=True)

# Plot: Price and Quantity Convergence
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

ax1.plot(combined_df['Step'], combined_df['Price'], color='blue', alpha=0.7, linewidth=1)
ax1.axhline(params.P_OPT, color='red', linestyle='--', linewidth=2, label=f'Optimal = {params.P_OPT:.2f}')
ax1.set_xlabel('Day')
ax1.set_ylabel('Price')
ax1.set_title('Price Convergence')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(combined_df['Step'], combined_df['Quantity'], color='green', alpha=0.7, linewidth=1)
ax2.axhline(params.Q_OPT, color='red', linestyle='--', linewidth=2, label=f'Optimal = {params.Q_OPT}')
ax2.set_xlabel('Day')
ax2.set_ylabel('Quantity')
ax2.set_title('Quantity Convergence')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Plot: Profit and Cumulative Regret
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

ax1.plot(combined_df['Step'], combined_df['Reward'], color='purple', alpha=0.7, linewidth=1)
ax1.axhline(params.PROFIT_OPT, color='red', linestyle='--', linewidth=2, label=f'Optimal = {params.PROFIT_OPT:.2f}')
ax1.set_xlabel('Day')
ax1.set_ylabel('Daily Profit')
ax1.set_title('Daily Profit')
ax1.legend()
ax1.grid(True, alpha=0.3)

cumulative_regret = combined_df['Regret'].cumsum()
ax2.plot(combined_df['Step'], cumulative_regret, color='red', linewidth=2)
ax2.set_xlabel('Day')
ax2.set_ylabel('Cumulative Regret')
ax2.set_title('Cumulative Regret')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Plot: Price-Quantity Coordination
fig, ax = plt.subplots(figsize=(12, 8))

scatter = ax.scatter(combined_df['Price'], combined_df['Quantity'], 
                     c=combined_df['Step'], cmap='viridis', 
                     alpha=0.6, s=50)

ax.scatter([params.P_OPT], [params.Q_OPT], color='red', s=500, 
           marker='*', label=f'Optimal (p={params.P_OPT:.2f}, q={params.Q_OPT})')

plt.colorbar(scatter, ax=ax, label='Day')
ax.set_xlabel('Price')
ax.set_ylabel('Quantity')
ax.set_title('Price-Quantity Coordination')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()